# End to End Image Classification Model

This notebook builds an end-to-end dog classification machine learning model using TensorFlow and TensorFlow Hub.

## 1. Problem Statement

Identifying Dog Breed

The goal is to develop a model that can identify the breed of a dog from a photograph.

## 2. Data

The data is collected from Kaggle: [Dog Breed Identification](https://www.kaggle.com/c/dog-breed-identification/data).

## 3. Model Evaluation

The model evaluation will be based on a file containing prediction probabilities for each dog breed.

## 4. Features

Some information about our dataset:
- We are dealing with unstructured data (images), so we will likely use deep learning/transfer learning techniques.
- The dataset contains over 10,000 images in the training set and over 10,000 images in the test set (these images are unlabeled as they are the ones we need to predict).

# Mounting Google Drive & Importing necessary libraries
1. Mounting google drive
2. Import libraries
3. Checking GPU
4. Init GPU
5. Wamup the GPU and CPU

## Mount Drive

In [ ]:
# Mount Google Drive to access data (if running on Google Colab)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import Libraries

In [107]:
# Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf


# Check versions
print(f"TensorFlow Version: {tf.__version__}")
print(f"TensorFlow Hub Version: {hub.__version__}")

2.15.0
0.16.1


## Checking GPU

In [ ]:
print("GPU", "Available" if tf.config.list_physical_devices("GPU") else "Not Available")

## Init GPU

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## Device warmup

In [ ]:
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

# Data Loading and Exploration
1. Loading Image Data
2. Displaying Sample Images
3. Exploring Image Metadata

## Loading & Exploring Image Data

In [ ]:
# Load the dataset
df_lab = pd.read_csv('drive/MyDrive/Deep_Learning/labels.csv')

In [ ]:
# Display dataset statistics
print(df_lab.describe())

In [ ]:
# Display the first few rows of the dataset
print(df_lab.head())

In [ ]:
# Display the value counts of breeds
breed_counts = df_lab['breed'].value_counts()
print(breed_counts)

In [ ]:
# Getting the median of count of images by breed
df_lab["breed"].value_counts().median()

In [ ]:
# Plotting grapph of count of images by breed
df_lab["breed"].value_counts().plot.bar(figsize=(20,10))

## Displaying Sample Images

In [ ]:
# Displaying a sample image
from IPython.display import Image, display

# Function to display a random image
def display_random_image(df, image_folder):
    random_index = np.random.randint(len(df))
    image_id = df["id"][random_index]
    breed = df["breed"][random_index]
    image_path = f"{image_folder}/{image_id}.jpg"
    display(Image(filename=image_path))
    print(f"Image ID: {image_id}, Breed: {breed}")

# Display a random image from the dataset
display_random_image(df_labels, "drive/MyDrive/Deep_Learning/train")

## Exploring Image Metadata

In [ ]:
# Display specific image by index
index = 8000
image_id = df_labels["id"][index]
breed = df_labels["breed"][index]
image_path = f"drive/MyDrive/Deep_Learning/train/{image_id}.jpg"
display(Image(filename=image_path))
print(f"Image ID: {image_id}, Breed: {breed}")


# Data Preparation
1. Extracting Labels
2. Checking Data Integrity
3. Converting Labels to Boolean Values
4. Preprocessing Images
5. Creating Data Pipeline
6. Vizualizing Data Batches

## Extracting Labels

In [ ]:
# Convert labels to numpy array
labels = df_labels["breed"].to_numpy()
print(labels)

## Checking Data Integrity

In [ ]:
# Ensure the number of labels matches the number of filenames
if len(labels) == len(df_labels["id"]):
    print("Number of labels matches number of filenames")
else:
    print("Numbers do not match")

# Check the length of labels
print(f"Number of labels: {len(labels)}")

## Converting Labels to Boolean Values

In [ ]:
# Get unique breed names
unique_breeds = np.unique(labels)
print(f"Unique breeds: {unique_breeds}")

# Convert labels to boolean values
bool_labels = [label == unique_breeds for label in labels]
print(f"Boolean labels: {bool_labels[:10]}")

# Verify the length of boolean labels
print(f"Number of boolean labels: {len(bool_labels)}")

### Cheking example boolean label and its occurence

In [ ]:
print(labels[0]) #original label
print(np.where(unique_breed==labels[0])) #index where label occured
print(bool_labels[0].argmax()) #index where label occurs in boolean array
print(bool_labels[0].astype(int)) #there will be 1 where the label occured

## Preprocessing Images
1. Loading and Resizing Images
2. Normalizing Images

In [ ]:
# define image size
img_size = 224

# Define function to preprocess images
def preprocess_image(image_path, img_size=img_size):
    # Read the image file
    image = tf.io.read_file(image_path)
    # Decode the image to a tensor
    image = tf.image.decode_jpeg(image, channels=3)
     #convert the color channel value form 0 - 255  to 0 - 1 values
    image = tf.image.convert_image_dtype(image, tf.float32)
    # resizing images
    image = tf.image.resize(image, size=[img_size, img_size])
    # Normalize the image to the range [0, 1]
    image = image / 255.0
    return image

# Test the function with a sample image
sample_image_path = f"drive/MyDrive/Deep_Learning/train/{df_labels['id'][0]}.jpg"
sample_image = preprocess_image(sample_image_path)
print(sample_image.shape)

## Creating Data Pipeline
1. Creating TensorFlow Dataset
2. Batching and Prefetching Data

In [ ]:
# Function to create a TensorFlow dataset from image paths and labels
def create_dataset(image_paths, labels, batch_size=32):
    # Create a dataset of image paths
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    
    # Function to load and preprocess images
    def load_and_preprocess_image(path, label):
        image = preprocess_image(path)
        return image, label
    
    # Map the load_and_preprocess_image function to the dataset
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # Shuffle, batch, and prefetch the dataset
    dataset = dataset.shuffle(buffer_size=len(image_paths)).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    return dataset

# Creating image paths
image_paths = [f"drive/MyDrive/Deep_Learning/train/{image_id}.jpg" for image_id in df_labels['id']]

# Creating the dataset
train_dataset = create_dataset(image_paths, bool_labels)

In [ ]:
# Function to return a tuple of (image, label)
def get_image_details(image_path, label):
    image = preprocess_image(image_path)
    return image, label

# Function to create batches of data
def create_data_batches(image_paths, labels, batch_size=32, valid_data=False):
    if valid_data:
        print("Creating validation data batches...")
    else:
        print("Creating training data batches...")
    
    # Create a dataset from the image paths and labels
    data = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    
    # Map the preprocessing function to the dataset
    data = data.map(get_image_details, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    # Batch and prefetch the dataset
    data = data.batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    return data

In [ ]:
# Example image paths and labels
image_paths = [f"drive/MyDrive/Deep_Learning/train/{image_id}.jpg" for image_id in df_labels['id']]

# Create training and validation data batches
train_data = create_data_batches(image_paths, bool_labels)
val_data = create_data_batches(image_paths, bool_labels, valid_data=True)

# Select one batch for visualization
one_batch_train_data = train_data.take(1)

## Visualizing Data Batches

In [ ]:
import matplotlib.pyplot as plt

# Function to visualize data batches
def visualize_data_batches(data_batch):
    # Iterate over the batch
    for images, labels in data_batch:
        # Plot each image in the batch
        plt.figure(figsize=(10, 10))
        for i in range(len(images)):
            plt.subplot(4, 4, i + 1)
            plt.imshow(images[i])
            plt.title(unique_breeds[np.argmax(labels[i])])
            plt.axis("off")
        plt.show()

# Visualize one batch of training data
visualize_data_batches(one_batch_train_data)

# Model Building, Training, and Evaluation
1. Create Model
2. Creating Model Callbacks
3. Training the Model
4. Making Predictions
5. Turning Predictions into Labels
6. Unbatching Validation Data
7. Plotting Predictions
8. Predicting Top 10 Images
9. Visualizing Both Graph and Image
10. Saving and Exporting the Model
11. Training a Big Model



## Create Model

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

# Setup input shape to the model
img_size = 224  # Example size, adjust as needed
input_shape = [None, img_size, img_size, 3]  # batch, height, width, color channels
output_shape = len(unique_breeds)

# Setup model URL from TensorFlow Hub (we're going to use already trained model)
model_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"

# Creating a function to build the model
def create_model(input_shape=input_shape, output_shape=output_shape, model_url=model_url):
    print("Building model with:", model_url)
    model = tf.keras.Sequential([
        hub.KerasLayer(model_url, input_shape=input_shape[1:]),
        tf.keras.layers.Dense(units=output_shape, activation="softmax")
    ])
    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=["accuracy"]
    )
    return model

model = create_model()
model.summary()

## Creating Model Callbacks

In [ ]:
import os
import datetime

# TensorBoard callback
def create_tensorboard_callback():
    logdir = os.path.join("drive/MyDrive/Deep_Learning/logs",
                          datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    return tf.keras.callbacks.TensorBoard(logdir)

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3)


## Training the Model

In [ ]:
num_epochs = 5  # Example value, adjust as needed

# Function to train the model
def train_model():
    """Trains a given model and returns the trained model"""
    model = create_model()
    tensorboard = create_tensorboard_callback()
    
    # Fitting the model
    model.fit(x=train_data,
              epochs=num_epochs,
              validation_data=val_data,
              validation_freq=1,
              callbacks=[tensorboard, early_stopping])
    
    return model

# Train the model
model = train_model()

## Making Predictions

In [ ]:
# Make prediction on validation data
predictions = model.predict(val_data, verbose=1)

# Example to view predictions
index = 0
print(predictions[index])
print(f"Max Value: {np.max(predictions[index])}")
print(f"Sum: {np.sum(predictions[index])}")
print(f"Max Index: {np.argmax(predictions[index])}")
print(f"Predicted Label: {unique_breeds[np.argmax(predictions[index])]}")

## Turning Predictions into Labels

In [ ]:
# Function to get predicted label
def get_pred_label(prediction_probabilities):
    """Turning the array of prediction probability into image label"""
    return unique_breeds[np.argmax(prediction_probabilities)]

pred_label = get_pred_label(predictions[21])
print(pred_label)

## Unbatching Validation Data

In [ ]:
# Function to unbatch data
def unbatch(data):
    images = []
    labels = []
    
    for image, label in data.unbatch().as_numpy_iterator():
        images.append(image)
        labels.append(unique_breeds[np.argmax(label)])
    return images, labels

val_images, val_labels = unbatch(val_data)
print(val_images[0], val_labels[0])


## Plotting Predictions

In [ ]:
import matplotlib.pyplot as plt

# Function to plot predictions
def plot_pred(prediction_probabilities, labels, images, n=0):
    pred_prob, true_label, image = prediction_probabilities[n], labels[n], images[n]
    pred_label = get_pred_label(pred_prob)
    
    plt.imshow(image)
    plt.xticks([])
    plt.yticks([])
    
    # Red for wrong and green for right prediction
    color = "green" if pred_label == true_label else "red"
    plt.title("{} {:2.0f}% {}".format(pred_label, np.max(pred_prob)*100, true_label), color=color)

# Visualizing predictions
plot_pred(prediction_probabilities=predictions, labels=val_labels, images=val_images, n=119)

## Predicting Top 10 Images

In [ ]:
# Function to plot top 10 predictions
def plot_pred_conf(prediction_probabilities, labels, unique_breed, n=1):
    pred_prob = prediction_probabilities[n]
    true_label = labels[n]
    
    # 10 prediction confidence index
    top_10_pred = pred_prob.argsort()[-10:][::-1]
    
    # 10 prediction confidence value
    top_10_value = pred_prob[top_10_pred]
    
    # 10 prediction label
    top_10_label = unique_breeds[top_10_pred]
    
    # Plotting
    top_plot = plt.bar(np.arange(len(top_10_label)), top_10_value, color='grey')
    plt.xticks(np.arange(len(top_10_label)), labels=top_10_label, rotation='vertical')
    
    # Coloring
    if true_label in top_10_label:
        top_plot[np.argmax(top_10_label == true_label)].set_color('green')
    
    plt.show()

# Example usage
plot_pred_conf(prediction_probabilities=predictions, labels=val_labels, unique_breeds=unique_breeds, n=14)

## Visualizing Both Graph and Image

In [ ]:
i_multiplier = 30
num_rows = 3
num_cols = 2
num_images = num_rows * num_cols
plt.figure(figsize=(10*num_cols, 5*num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_pred(prediction_probabilities=predictions, labels=val_labels, images=val_images, n=i+i_multiplier)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_pred_conf(prediction_probabilities=predictions, labels=val_labels, unique_breeds=unique_breeds, n=i+i_multiplier)

plt.tight_layout(h_pad=1.0)
plt.show()

## Saving and Exporting the Model

In [ ]:
# Function to save the model
def save_model(model, suffix=None):
    model_dir = os.path.join("drive/MyDrive/Deep_Learning/models", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    model_path = model_dir + "-" + suffix + ".h5"
    model.save(model_path)
    return model_path

# Function to load a trained model
def load_model(model_path):
    print(f"Loading Saved Model: {model_path}")
    model = tf.keras.models.load_model(model_path, custom_objects={"KerasLayer": hub.KerasLayer})
    return model

# Saving the model
save_model(model, suffix="1000_images_mobileNet_Adam")

# Loading the trained model
loaded_model = load_model("drive/MyDrive/Deep_Learning/models/20240530-17441717091054-1000_images_mobileNet_Adam.h5")

# Evaluating loaded model
loaded_model.evaluate(val_data)

## Training a Big Model

In [ ]:
# Create data batch with the full dataset
full_data = create_data_batches(image_paths, bool_labels)
print("Creating training data batches...")

# Create a model for full dataset
full_model = create_model()
full_tensorboard_callback = create_tensorboard_callback()
full_model_early_stopping = tf.keras.callbacks.EarlyStopping(monitor="accuracy", patience=3)
full_model.fit(x=full_data, epochs=num_epochs, callbacks=[full_tensorboard_callback, full_model_early_stopping])

# Saving full model
save_model(full_model, suffix="full_image_reco_model")